## Example of Batch Processing Using Folder Name with Variable Tags

This example demonstrates how to batch process images using folder name as multiple variable tags. Each folder represents a different set of experimental conditions, and these conditions are extracted from the folder name as variables.Each variable intended to be represented a value in the specified column in the resulting output table. This variables in the name of the folder are delineated by underscores (`_`). The script then used the function to processes paired images within the folder and aggregates the results into master DataFrames, which are then saved as CSV files.

## Explanation

1. **Setup Output Directories**:
    - The script first sets up directories for storing the output data. It creates an `Outputs` folder and a subfolder `Output_images` to store the results and images respectively.

2. **Initialize Data Structures**:
    - Master DataFrames (`master_results` and `master_reports`) and other necessary data structures are initialized to collect the results from each processed folder.

3. **Iterate Over Folders**:
    - The script iterates over each subfolder in the `global_path` directory. Each subfolder represents a different date of experiments.

4. **Process Each Subfolder**:
    - For each subfolder (representing a specific experiment), it extracts variables (such as cells, treatment, concentration) from the folder name. These variables are expected to be separated by underscores (`_`).

5. **Process Paired Images**:
    - The function `in_folder_loop` is called to process the images within each subfolder. It uses the selected analysis function to generate results and reports.

6. **Assign Identifiers**:
    - The extracted variables (cells, treatment, concentration, date) and the name of the selected analysis function are added as columns to the results and reports DataFrames.




In [1]:
import pandas as pd
from pathlib import Path
import natsort
from FLIM_functions import analyze_single_fitted_channel,generate_small_report_fig,analyze_single_fastflim_channel
from helper_functions import in_folder_loop

In [7]:
#global_path=Path(r"F:\SP8_FLIM\Treatments\Aldometanib")
global_path=Path(r"G:\SP8_FLIM\sMol_experiment")

In [8]:
selected_analysis_function = analyze_single_fitted_channel

In [9]:
import pandas as pd
from pathlib import Path
import natsort


# Create the folder where all the outputs will be collected 
output_folder_path = global_path / 'Outputs'
output_folder_path.mkdir(exist_ok=True)

outputimage_folder_path = output_folder_path / 'Output_images'
outputimage_folder_path.mkdir(exist_ok=True)

# Initialize master DataFrames and other data structures outside the loop
master_results = pd.DataFrame([])
master_reports = pd.DataFrame([])
report_images = []


# Iterate over all folders in the global_path directory
for day_path in global_path.iterdir():
    if day_path.is_dir() and day_path.stem != 'Outputs':
        
        date = day_path.stem
        print(f"Processing date folder: {date}")
        
        result_per_date = pd.DataFrame()
        
        # level of the folder with analisys, each variable is a 'tag' between _ 
        #For different experiment the columns may be modified depending on the design 
        
        for cell_treatment_path in day_path.iterdir():
            if cell_treatment_path.is_dir(): 
                
                cell_treatment = cell_treatment_path.stem
                
                # Split the folder name to extract cells, age, treatment, and concentration
                try:
                    cells, transfection, treatment  = cell_treatment.split('_')
                
                except ValueError:
                    print(f"Skipping folder with unexpected name format: {cell_treatment}")
                    continue
                
                print(f"Cells: {cells}")
                print(f"Transfection: {transfection}")
                print(f"treatment: {treatment}")
                print(f"  Processing cell_treatment folder: {cell_treatment}")

                result_per_folder,report_per_folder=in_folder_loop(cell_treatment_path,selected_analysis_function)
                
                # identifiction 
                result_per_folder['cells'] = cells
                result_per_folder['transfection'] = transfection
                result_per_folder['treatment'] = treatment

                result_per_folder['Date'] = date
                result_per_folder['selected_analysis'] = selected_analysis_function.__name__

                report_per_folder['cells'] = cells
                report_per_folder['treatment'] = treatment
                report_per_folder['transfection'] = transfection

                report_per_folder['Date'] = date
                report_per_folder['selected_analysis'] =selected_analysis_function.__name__

                # Concatenate the local table to the master DataFrame
                master_results = pd.concat([master_results, result_per_folder])
                master_reports = pd.concat([master_reports, report_per_folder])



# Save the master DataFrame as a CSV file
master_results.to_csv( "master_table.csv")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv("master_reports.csv")


Processing date folder: 2024-09-11
Cells: cl1
Transfection: pcdna
treatment: cq
  Processing cell_treatment folder: cl1_pcdna_cq
Cells: cl1
Transfection: pcdna
treatment: dmso
  Processing cell_treatment folder: cl1_pcdna_dmso
Cells: cl1
Transfection: pcdna
treatment: fluox
  Processing cell_treatment folder: cl1_pcdna_fluox
Cells: cl1
Transfection: pcdna
treatment: prop
  Processing cell_treatment folder: cl1_pcdna_prop
Cells: cl1
Transfection: pcdna
treatment: res
  Processing cell_treatment folder: cl1_pcdna_res
Cells: cl1
Transfection: vmat
treatment: imi
  Processing cell_treatment folder: cl1_vmat_imi
Cells: cl1
Transfection: vmat
treatment: prop
  Processing cell_treatment folder: cl1_vmat_prop
Cells: cl1
Transfection: vmat
treatment: res
  Processing cell_treatment folder: cl1_vmat_res
Cells: cl1
Transfection: vmat
treatment: fluox
  Processing cell_treatment folder: cl1_vmat_fluox
Cells: cl1
Transfection: pcdna
treatment: imi
  Processing cell_treatment folder: cl1_pcdna_imi
P

In [10]:
master_reports

,Mean_fluor_intensity,Mean_tau,Mean_STD,Number_of_objects,Initial_number_of_objects,%_of_object_remained,File Name,cells,treatment,transfection,Date,selected_analysis
0,506.307193,1013.243908,41.753698,50,93,53.763441,1f,cl1,cq,pcdna,2024-09-11,analyze_single_fitted_channel
0,481.954647,1066.618447,41.945314,62,85,72.941176,2f,cl1,cq,pcdna,2024-09-11,analyze_single_fitted_channel
0,361.820145,1001.294082,22.673406,9,79,11.392405,3f,cl1,cq,pcdna,2024-09-11,analyze_single_fitted_channel
0,351.244159,1087.484785,22.194405,4,89,4.494382,4f,cl1,cq,pcdna,2024-09-11,analyze_single_fitted_channel
0,464.732386,902.046525,35.144006,68,117,58.119658,5f,cl1,cq,pcdna,2024-09-11,analyze_single_fitted_channel
...,...,...,...,...,...,...,...,...,...,...,...,...
0,501.384156,1025.289180,28.668745,103,120,85.833333,1f,cl2,imi,pcdna,2024-09-15,analyze_single_fitted_channel
0,466.008590,1093.602374,32.807463,93,123,75.609756,2f,cl2,imi,pcdna,2024-09-15,analyze_single_fitted_channel
0,414.635513,1065.564088,35.267824,89,157,56.687898,3f,cl2,imi,pcdna,2024-09-15,analyze_single_fitted_channel
0,539.727879,1052.860201,32.183790,48,76,63.157895,4f,cl2,imi,pcdna,2024-09-15,analyze_single_fitted_channel


In [11]:
print("Saving master table...")
# Save the master DataFrame as a CSV file
master_results.to_csv(output_folder_path / "master_table.csv")

print("Saving table of reports...")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv(output_folder_path / "master_reports.csv")

print("Processing complete!")

Saving master table...
Saving table of reports...
Processing complete!
